In [260]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
from dataset import InputFeatures, load_and_cache_examples
import dataset
import functionsAval as f

In [261]:
f = reload(f)
reload(dataset)
reload(eval)

path_model=r'.\model-multilabel-io\model'

In [262]:
tags, tokens = f.predictBERTNER_multi_IO(['The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induced within several hours even in the presence of cycloheximide .'], path_model)
print('-----------')
print(tags)
print(tokens)

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
-----------
[['O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['the', 'hb24', 'mrna', 'was', 'absent', 'or', 'present', 'at', 'low', 'levels', 'in', 'normal', 'b', 'and', 't', 'lymphocytes', ';', 'however', ',', 'with', 'the', 'appropriate', 'activation', 'signal', 'hb24', 'mrna', 'was', 'induced', 'within', 'several', 'hours', 'even', 'in', 'the', 'presence', 'of', 'cycloheximide', '.']]


In [432]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        print('ERROO')
        raise
        return None
#save_obj('dicEntidadesTesteGenia', dicEntidades)

def loadSentencesTest():
    # gabarito = dicSentences_new.pkl
    #dicSentences_train = load_obj('dic_sentencesTrain.pkl')
    #dicSentences_train[32]
    # gabarito = dicSentences_new.pkl
    print('Pegando sentencas de teste gabarito: dic_sentencesTest.pkl')
    dicSentences_new_test = load_obj('dicEntidadesTesteGenia')
    return dicSentences_new_test

#dicSentences_new_test = loadSentencesTest()
#dicSentences_new_test


In [436]:
dicSentences_new_test[0]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 [['methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  ['homeobox', [20], 'DNA'],
  ['CAX repeat', [23, 24], 'DNA'],
  ['open reading frame', [28, 29, 30], 'DNA']]]

In [435]:
#BATCH=100
BATCH=2000
dicSentences_new_test = loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        #tokens = [tok[0] for tok in tokens]
        #sentences.append(' '.join(tokens).strip())
        sentences.append(tokens)

Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
1854
[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .', [['methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], ['homeobox', [20], 'DNA'], ['CAX repeat', [23, 24], 'DNA'], ['open reading frame', [28, 29, 30], 'DNA']]]
numero de sentencas no total: 1854


In [410]:
sentences[:2]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 ' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .']

In [437]:
dicSentences_new_test[3]

[' Characterization of HB24 expression in lymphoid and select developing tissues was performed by in situ hybridization .',
 [['HB24', [2], 'DNA']]]

In [423]:
def getDicPredictions(tags, tokens):
    listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

    dic_predictions = {}
    i=0
    for tags_sentence, tokens_sentence in zip(tags, tokens):
        listaEntidades=[]
        isEnt=0
        labelAnterior=''
        entidade=''
        label=''
        #listaTags
        for classe in listaTags:
            num_token=0
            indices_entidade = list()
            for tag, token in zip(tags_sentence, tokens_sentence):
                #print(token, tag)
                labels = tag.split('-')
                if tag!='O' and classe in labels:
                    isEnt=1
                    indices_entidade.append(num_token)
                    #print('---label-->:', label)
                    #print('---labelAnterior-->:', labelAnterior)
                    #if label == labelAnterior:
                    if labelAnterior in classe:
                        # continuacao
                        entidade = entidade+' '+token
                    elif labelAnterior=='': # primeiro token da entidade
                        entidade=token
                        labelAnterior=classe
                    else: # mudou de entidade
                        listaEntidades.append([entidade, indices_entidade, labelAnterior])
                        indices_entidade=list()
                        entidade=token
                    labelAnterior = classe
                else:
                    if isEnt==1:
                        if entidade:
                            #print('---label-->:', label)
                            listaEntidades.append([entidade, indices_entidade, classe])
                            indices_entidade=list()
                    entidade=''
                    labelAnterior=''
                    isEnt=0
                    label=''
                num_token=num_token+1

        if len(listaEntidades)>0:
            dic_predictions[i]=['FRASE', listaEntidades]
        else:
            dic_predictions[i]=['FRASE', []]
        i=i+1

    return dic_predictions
dic_predictions = getDicPredictions(tags, tokens)
#dic_predictions

In [413]:
tags, tokens = f.predictBERTNER_multi_IO(sentences, path_model)
dic_predictions = getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
['FRASE', [[' methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], [' homeobox', [20], 'DNA'], [' cax repeat', [23, 24], 'DNA'], [' open reading frame', [28, 29, 30], 'DNA']]]
len(dic_predictions): 1854


In [414]:
dic_predictions[0]

['FRASE',
 [[' methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  [' homeobox', [20], 'DNA'],
  [' cax repeat', [23, 24], 'DNA'],
  [' open reading frame', [28, 29, 30], 'DNA']]]

In [415]:
dicSentences_new_test[0]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 [['methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  ['homeobox', [20], 'DNA'],
  ['CAX repeat', [23, 24], 'DNA'],
  ['open reading frame', [28, 29, 30], 'DNA']]]

In [445]:
i=0
for a, b in zip(dic_predictions.items(),dicSentences_new_test.items()):
    print('--sentenca (predito vs real):--')
    aa=a[1][1]
    bb=b[1][1]
    print(aa)
    print(bb)
    i=i+1
    if i>100:
        break

--sentenca:--
[[' methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], [' homeobox', [20], 'DNA'], [' cax repeat', [23, 24], 'DNA'], [' open reading frame', [28, 29, 30], 'DNA']]
[['methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], ['homeobox', [20], 'DNA'], ['CAX repeat', [23, 24], 'DNA'], ['open reading frame', [28, 29, 30], 'DNA']]
--sentenca:--
[[' homeodomain', [2], 'DNA'], [' hb24', [4], 'DNA'], [' drosophila homeodomain', [31, 32], 'DNA'], [' h2 . 0', [34, 35, 36], 'DNA']]
[['homeodomain', [2], 'DNA'], ['HB24', [4], 'DNA'], ['homeodomain', [32], 'DNA'], ['Drosophila homeodomain', [31, 32], 'DNA']]
--sentenca:--
[[' hb24', [1], 'DNA'], [' hb24', [24], 'DNA'], [' hb24 mrna', [1, 2], 'RNA'], [' hb24 mrna', [24, 25], 'RNA']]
[['HB24 mRNA', [1, 2], 'RNA'], ['HB24 mRNA', [24, 25], 'RNA'], ['T lymphocytes', [14, 15], 'cell_type']]
--sentenca:--
[[' hb24', [2], 'protein']]
[['HB24', [2], 'DNA']]
--sentenca:--
[]
[]
--sentenca:

In [416]:
dicSentences_new_test[0][1][0]

['methionine codon - initiated open reading frame',
 [4, 5, 6, 7, 8, 9, 10],
 'DNA']

In [417]:
def AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH):

    region_true_list, region_pred_list = list(), list() # labels
    region_true_count, region_pred_count = 0, 0 # contagem
    numErro1=0
    numErro2=0

    for i in range(0, BATCH, 1):
        if i<len(dicSentences_new_test):
        #if 1==1:           
            #print('\n---Label vs predicao--------')
            #print('frase:', dicSentences_new_test[i][0])
            #print('i:', i)
            #print(dicSentences_new_test[i][1])
            #print(dic_predictions_all[i][1])
            entidades_gabarito = dicSentences_new_test[i][1]
            try:
                #print(dic_predictions_all[i])
                entidades_preditas = dic_predictions_all[i][1]
            except:
                print('erro, caiu no except na AvalFinal:', str(i))
                raise
            #print('entidades_preditas:', entidades_preditas)
            for entidade_gabarito in entidades_gabarito:
                #entidade_gabarito=entidade_gabarito[0]
                indices_gabarito = entidade_gabarito[1]
                tag_gabarito = entidade_gabarito[2]
                region_true_count=region_true_count+1
                region_true_list.append(tag_gabarito)
                # ver se previu essa entidade
                previu=0
                for entidade_predita in entidades_preditas:
                    indices_predita = entidade_predita[1]
                    tag_predita = entidade_predita[2]
                    if indices_predita == indices_gabarito:
                        region_pred_list.append(tag_predita)
                        previu=1
                        #print('achou')
                        if tag_predita !='O':
                            region_pred_count=region_pred_count+1
                        break
                if previu==0:
                    numErro1=numErro1+1
                    region_pred_list.append('O')

            # agora o contrario, ver o q previu mas nao era

            for entidade_predita in entidades_preditas:
                indices_predita = entidade_predita[1]
                tag_predita = entidade_predita[2]
                # ver se a entidade prevista existe ou é FP
                existe=0
                for entidade_gabarito in entidades_gabarito:
                    #entidade_gabarito=entidade_gabarito[0]
                    indices_gabarito = entidade_gabarito[1]
                    if indices_predita == indices_gabarito:
                        existe=1
                        break
                if existe==0:
                    numErro2 = numErro2+1
                    region_true_list.append('O')
                    region_pred_list.append(tag_predita)

    #print('===resultados====')
    #print('region_true_list:', region_true_list)
    #print('region_pred_list:', region_pred_list)

    print('numErro1:', numErro1)
    print('numErro2:', numErro2)
    #print(classification_report(region_true_list, region_pred_list, labels=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], target_names=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], digits=6))
    print(classification_report(region_true_list, region_pred_list, digits=6))

    return region_true_list, region_pred_list

region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions, BATCH)


numErro1: 1567
numErro2: 1124
              precision    recall  f1-score   support

         DNA   0.709677  0.691935  0.700694      1240
           O   0.000000  0.000000  0.000000      1124
         RNA   0.711712  0.724771  0.718182       109
   cell_line   0.752137  0.604119  0.670051       437
   cell_type   0.728374  0.698176  0.712955       603
     protein   0.760507  0.685276  0.720934      3063

    accuracy                       0.565845      6576
   macro avg   0.610401  0.567379  0.587136      6576
weighted avg   0.616622  0.565845  0.589734      6576



In [418]:
#dic_predictions[5]

In [419]:
region_true_list[0:30]

['DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'O',
 'RNA',
 'RNA',
 'cell_type',
 'O',
 'O',
 'DNA',
 'DNA',
 'cell_type',
 'DNA',
 'cell_line',
 'protein',
 'protein',
 'cell_type',
 'cell_type',
 'cell_type',
 'protein',
 'cell_type',
 'protein',
 'cell_line',
 'cell_line',
 'cell_line']

In [420]:
region_pred_list[0:30]

['DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'O',
 'DNA',
 'DNA',
 'RNA',
 'RNA',
 'O',
 'DNA',
 'DNA',
 'protein',
 'protein',
 'cell_type',
 'DNA',
 'cell_line',
 'protein',
 'protein',
 'cell_type',
 'cell_type',
 'cell_type',
 'protein',
 'cell_type',
 'protein',
 'O',
 'O',
 'O']

In [421]:
dicSentences_new_test[0]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 [['methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  ['homeobox', [20], 'DNA'],
  ['CAX repeat', [23, 24], 'DNA'],
  ['open reading frame', [28, 29, 30], 'DNA']]]

In [422]:
dic_predictions[0]

['FRASE',
 [[' methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  [' homeobox', [20], 'DNA'],
  [' cax repeat', [23, 24], 'DNA'],
  [' open reading frame', [28, 29, 30], 'DNA']]]

In [408]:
#BATCH - 100
#region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions, BATCH)
#print(confusion_matrix(region_true_list, region_pred_list))

numErro1: 96
numErro2: 54
              precision    recall  f1-score   support

         DNA   0.760000  0.684685  0.720379       111
           O   0.000000  0.000000  0.000000        54
         RNA   0.750000  0.600000  0.666667         5
   cell_line   0.714286  0.588235  0.645161        34
   cell_type   0.947368  0.818182  0.878049        22
     protein   0.795455  0.686275  0.736842       153

    accuracy                       0.585752       379
   macro avg   0.661185  0.562896  0.607850       379
weighted avg   0.672671  0.585752  0.626081       379

[[ 76  29   0   0   0   6]
 [ 24   0   1   8   0  21]
 [  0   2   3   0   0   0]
 [  0  14   0  20   0   0]
 [  0   4   0   0  18   0]
 [  0  47   0   0   1 105]]
